In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [2]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.8 MB/s eta 0:00:00


In [32]:
# 라이브러리
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pickle
import random

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from torchsummary import summary

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from peft import get_peft_model, LoraConfig, TaskType

In [4]:
# OS 충돌 방지
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Matplotlib 커널 종료 방지
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
# 시드 고정
def fixSEED(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

fixSEED(seed=42)

In [6]:
# 디렉토리 설정
dir = 'Model'
path = '/content/drive/MyDrive/Projects/' + dir
os.chdir(path)

In [7]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### 1. 데이터 불러오기 및 가공

In [8]:
# pickle 불러오기
with open("totalData.pkl", "rb" ) as file:
    data = pickle.load(file)

In [9]:
# 확인
data.head()

,partNum,newsID,newsCategory,newsSubcategory,newsContent,processLevel,processPattern,contentList,sentenceCount,processSentencenum,useType,wordCount
0,P2,LC_M02_551626,생활&문화,NaN,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,하,99,"[손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다., 어떤 이에게는 ...",20,3,0,276
1,P2,LC_M02_551627,생활&문화,NaN,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,하,99,[Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이...,16,3,0,204
2,P2,LC_M02_551629,생활&문화,NaN,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,하,99,[요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들...,27,3,0,285
3,P2,LC_M02_551631,생활&문화,NaN,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,하,99,[생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가...,17,4,0,446
4,P2,LC_M02_551632,생활&문화,NaN,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,하,99,"[자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다., 직원들이 지역 ...",16,2,0,196


In [10]:
# 레이블 값 변경하기
data['Label'] = np.where(data['useType'] == 0, 1, 0)

In [11]:
# 확인
data.head()

,partNum,newsID,newsCategory,newsSubcategory,newsContent,processLevel,processPattern,contentList,sentenceCount,processSentencenum,useType,wordCount,Label
0,P2,LC_M02_551626,생활&문화,NaN,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,하,99,"[손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다., 어떤 이에게는 ...",20,3,0,276,1
1,P2,LC_M02_551627,생활&문화,NaN,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,하,99,[Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이...,16,3,0,204,1
2,P2,LC_M02_551629,생활&문화,NaN,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,하,99,[요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들...,27,3,0,285,1
3,P2,LC_M02_551631,생활&문화,NaN,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,하,99,[생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가...,17,4,0,446,1
4,P2,LC_M02_551632,생활&문화,NaN,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,하,99,"[자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다., 직원들이 지역 ...",16,2,0,196,1


In [12]:
# 앞, 뒤 n개의 문장 추출
def extract_sentences(row):
    # 비낚시성
    if row['Label'] == 0:
        front_sentences = row['contentList'][:3]
        back_sentences = row['contentList'][-3:]
    # 낚시성
    else:
        if row['processSentencenum'] == 1:
            front_sentences = row['contentList'][:2]
            back_sentences = row['contentList'][-2:]
        else:
            front_sentences = row['contentList'][:3]
            back_sentences = row['contentList'][-3:]

    return ' '.join(front_sentences), ' '.join(back_sentences)

In [13]:
new_data = data.copy()
del data

In [14]:
# 정해진 logic에 따라 앞뒤 문장 추출
new_data['frontContent'], new_data['backContent'] = zip(*new_data.apply(extract_sentences, axis=1))

In [15]:
# 결과 확인
new_data['frontContent'][0]

'손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분과 지위를 상징하고 또 다른 이에게는 패션과 개성을 상징한다. 여기에 가난을 없애고 목마름을 해소시키며 암을 예방해준다면 그리고 가격이 40달러에 불과하다면 당신은 이 손목시계를 구매할 의향이 있는가'

In [16]:
new_data.head()

,partNum,newsID,newsCategory,newsSubcategory,newsContent,processLevel,processPattern,contentList,sentenceCount,processSentencenum,useType,wordCount,Label,frontContent,backContent
0,P2,LC_M02_551626,생활&문화,NaN,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,하,99,"[손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다., 어떤 이에게는 ...",20,3,0,276,1,손목시계는 이제 시간을 알려주는 도구 이상의 의미를 갖고 있다. 어떤 이에게는 신분...,이 제품의 진정한 의미는 당신의 구매가 다른 사람에게 희망이 되고 살기 좋은 세상으...
1,P2,LC_M02_551627,생활&문화,NaN,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,하,99,[Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이...,16,3,0,204,1,Ruth Noguerón와 Amanda Stevens는 WRI에서 환경 영향을 이해...,그들은 라벨을 보거나 혹은 공급사에 직접 물어보는 방식을 통해서라도 이런 정보를 가...
2,P2,LC_M02_551629,생활&문화,NaN,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,하,99,[요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들...,27,3,0,285,1,요즘 대부분 소비자들은 식료품을 살 때마다 포장 겉면의 영양정보 표기를 자세히 들여...,우리 식품의약품안전처는 이런 흐름에 어떻게 반응할까. 우리 식료품 영양성분표시는 소...
3,P2,LC_M02_551631,생활&문화,NaN,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,하,99,[생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가...,17,4,0,446,1,생리대를사지못하는저소득층소녀들이깔창이나휴지등을생리대로쓰는사연이널리알려져많은사람이가슴...,예상대로진행된다면내년6월쯤국내첫월경컵이출시될것이다. 그러나 크라우드 펀딩의성공과관계...
4,P2,LC_M02_551632,생활&문화,NaN,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,하,99,"[자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다., 직원들이 지역 ...",16,2,0,196,1,자원봉사는 일반적으로 기업의 사회적 책임의 주요 구성 요소이다. 직원들이 지역 사회...,안전 수칙은 비상사태가 발생했을 때 자원봉사자가 따라야 할 과정과 절차를 의미한다....


### 2. 학습/평가용으로 나누기

In [17]:
# 먼저 test 확정
train_val_data, test_data = train_test_split(new_data, test_size=0.2, random_state=42)

In [18]:
# train / validation
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=42)

In [19]:
# 크기 확인
train_n = len(train_data)
val_n = len(val_data)
test_n = len(test_data)

print('학습용 데이터 개수 : {}개 \n검증용 데이터 개수 : {}개 \n평가용 데이터 개수 : {}개'.format(train_n, val_n, test_n))

학습용 데이터 개수 : 157390개 
검증용 데이터 개수 : 52464개 
평가용 데이터 개수 : 52464개


In [20]:
del new_data

### 3. Dataset, Dataloader 정의

⭐️ 데이터셋에서 토큰화를 진행 -> 모델에서 input_ids와 attention_mask를 입력 받아 처리

In [21]:
class BaitNewsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        front_text = self.data.iloc[idx]['frontContent']
        back_text = self.data.iloc[idx]['backContent']
        label = self.data.iloc[idx]['Label']

        # 앞 부분 + 뒷 부분
        total_text = front_text + back_text

        inputs = self.tokenizer(total_text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        return input_ids, attention_mask, torch.tensor(label, dtype=torch.float32)


### 4. 모델 정의

In [42]:
# 파라미터 개수 확인
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable Params: {trainable_params} \nAll params: {all_param}")
    print('-'*50)
    print(f"Trainable(%): {round(100 * trainable_params / all_param, 5)}")

In [43]:
# LoRA가 적용된 모델 정의
class BaitNewsDetectBERT(nn.Module):
    def __init__(self, model_checkpoint, lora_r=4, lora_alpha=1, lora_dropout=0.1):
        super(BaitNewsDetectBERT, self).__init__()

        # Pre-trained Model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

        # LoRA Config
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            target_modules=["query", "value"],  # Apply LoRA to the attention layers
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout
        )

        # Apply LoRA to Pre-trained Layers
        self.model = get_peft_model(self.model, lora_config)

        # Classifier Layer
        self.classifier = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask).logits
        logits = self.classifier(outputs)

        return logits

모델 구성 참고 : https://huggingface.co/docs/peft/developer_guides/custom_models

### 5. 학습, 검증용 함수

In [44]:
# 학습 과정 평가 함수
def train_model(model, train_dataloader, criterion, optimizer, acc_metric, f1_metric, device):
    model.train()

    train_loss = 0
    train_acc = 0
    train_f1 = 0

    for batch in tqdm(train_dataloader, desc='Training'):
        input_ids, attention_mask, y = batch
        input_ids, attention_mask, y = input_ids.to(device), attention_mask.to(device), y.to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += acc_metric(outputs.squeeze(), y).item()
        train_f1 += f1_metric(outputs.squeeze(), y).item()

    return train_loss / len(train_dataloader), train_acc / len(train_dataloader), train_f1 / len(train_dataloader)

In [45]:
# 검증 평가 함수
def valid_model(model, valid_dataloader, criterion, acc_metric, f1_metric, device):
    model.eval()

    valid_loss = 0
    valid_acc = 0
    valid_f1 = 0

    for batch in tqdm(valid_dataloader, desc='Validating'):
        input_ids, attention_mask, y = batch
        input_ids, attention_mask, y = input_ids.to(device), attention_mask.to(device), y.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(), y)

        valid_loss += loss.item()
        valid_acc += acc_metric(outputs.squeeze(), y).item()
        valid_f1 += f1_metric(outputs.squeeze(), y).item()

    return valid_loss / len(valid_dataloader), valid_acc / len(valid_dataloader), valid_f1 / len(valid_dataloader)

In [46]:
# 학습 진행 함수
def Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, acc_metric, f1_metric, device, num_epochs, wait_for_es):
    train_loss_history, train_acc_history, train_f1_history = [], [], []
    valid_loss_history, valid_acc_history, valid_f1_history = [], [], []

    best_loss = float('inf')
    count = 0
    best_acc = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        print('-' * 80)
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 80)

        train_loss, train_acc, train_f1 = train_model(model, train_dataloader, criterion, optimizer, acc_metric, f1_metric, device)
        valid_loss, valid_acc, valid_f1 = valid_model(model, valid_dataloader, criterion, acc_metric, f1_metric, device)

        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)
        train_f1_history.append(train_f1)

        valid_loss_history.append(valid_loss)
        valid_acc_history.append(valid_acc)
        valid_f1_history.append(valid_f1)

        elapsed_time = time.time() - start_time
        print(f'[Train] \t Loss: {train_loss:.4f} \t Acc: {train_acc:.4f} \t F1: {train_f1:.4f}')
        print(f'[Valid] \t Loss: {valid_loss:.4f} \t Acc: {valid_acc:.4f} \t F1: {valid_f1:.4f}')
        print(f'(Epoch {epoch+1} complete in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s)')

        if valid_acc > best_acc:
            best_acc = valid_acc
            torch.save(model.state_dict(), f'./CustomRoBERTa_epoch_{epoch+1}.pth')

        if valid_loss < best_loss:
            best_loss = valid_loss
            count = 0
        else:
            count += 1
            if count >= wait_for_es:
                print(f'-----<Early Stopping at Epoch {epoch+1}>-----')
                break

    return train_loss_history, train_acc_history, train_f1_history, valid_loss_history, valid_acc_history, valid_f1_history

### 6. 학습 진행

In [47]:
# 모델 체크포인트와 파라미터
batch_size = 32
num_labels = 2
max_length = 64

model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [48]:
# 데이터셋 정의
train_dataset = BaitNewsDataset(train_data, tokenizer)
val_dataset = BaitNewsDataset(val_data, tokenizer)
test_dataset = BaitNewsDataset(test_data, tokenizer)

In [49]:
# 데이터로더 정의
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

next(iter(train_dataloader))

[tensor([[    0,  3858,  2052,  ...,  2173,   558,     2],
         [    0,  3708,  5731,  ...,  2371,  2062,     2],
         [    0,  5443,   717,  ...,  5456, 21982,     2],
         ...,
         [    0,  3982,  2073,  ...,  2069,  3758,     2],
         [    0,  6910,  2116,  ...,  2595,  2052,     2],
         [    0,  9128, 11079,  ...,  2069,  5778,     2]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
         1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1.])]

In [50]:
model = BaitNewsDetectBERT(model_checkpoint)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BaitNewsDetectBERT(
  (model): PeftModelForSequenceClassification(
    (base_model): LoraModel(
      (model): RobertaForSequenceClassification(
        (roberta): RobertaModel(
          (embeddings): RobertaEmbeddings(
            (word_embeddings): Embedding(32000, 1024, padding_idx=1)
            (position_embeddings): Embedding(514, 1024, padding_idx=1)
            (token_type_embeddings): Embedding(1, 1024)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): RobertaEncoder(
            (layer): ModuleList(
              (0-23): 24 x RobertaLayer(
                (attention): RobertaAttention(
                  (self): RobertaSelfAttention(
                    (query): lora.Linear(
                      (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1,

In [51]:
# LoRA 적용 후 훈련 가능한 파라미터 개수
print_trainable_parameters(model)

Trainable Params: 1445123 
All params: 338103557
--------------------------------------------------
Trainable(%): 0.42742


In [52]:
# 손실함수와 옵티마이저, 그리고 Metrics
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
acc_metric = BinaryAccuracy().to(device)
f1_metric = BinaryF1Score().to(device)

In [53]:
# 학습 하이퍼파라미터
num_epochs = 1
wait_for_es = 10

In [54]:
# 학습 진행
train_loss_history, train_acc_history, train_f1_history, valid_loss_history, valid_acc_history, valid_f1_history = Trainer(model,
                                                                                                                           train_dataloader,
                                                                                                                           valid_dataloader,
                                                                                                                           criterion,
                                                                                                                           optimizer,
                                                                                                                           acc_metric,
                                                                                                                           f1_metric,
                                                                                                                           device,
                                                                                                                           num_epochs,
                                                                                                                           wait_for_es)

--------------------------------------------------------------------------------
Epoch 1/1
--------------------------------------------------------------------------------


Validation: 100%|██████████| 1640/1640 [10:42<00:00,  2.55it/s]

[Train] 	 Loss: 0.6825 	 Acc: 0.5529 	 F1: 0.5069
[Valid] 	 Loss: 0.6657 	 Acc: 0.5920 	 F1: 0.5288
(Epoch 1 complete in 75m 39s)


RuntimeError: Parent directory ./model does not exist.